# Various performance tips and helpful tools

## Code inspection macros

We can inspect the steps in the compilation process of a given function call all the way down to LLVM, thanks to a couple of macros. These can come in really handy when debugging performance issues.

In [1]:
function foo(x)
    x^2 + 3x - 1
end

foo (generic function with 1 method)

In [2]:
foo(1)

3

In [3]:
@code_lowered foo(1)

CodeInfo(
1 ─ %1  = Main.:-
│   %2  = Main.:+
│   %3  = Main.:^
│   %4  = Core.apply_type(Base.Val, 2)
│   %5  = (%4)()
│   %6  = Base.literal_pow(%3, x, %5)
│   %7  = Main.:*
│   %8  = (%7)(3, x)
│   %9  = (%2)(%6, %8)
│   %10 = (%1)(%9, 1)
└──       return %10
)

In [4]:
@code_typed foo(1)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
│   %2 = Base.mul_int(3, x)::Int64
│   %3 = Base.add_int(%1, %2)::Int64
│   %4 = Base.sub_int(%3, 1)::Int64
└──      return %4
) => Int64

In [5]:
@code_warntype foo(1)

MethodInstance for foo(::Int64)
  from foo(x) @ Main ~/Documents/Talks/CeCI/CECI-Julia-for-HPC/code/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W2sZmlsZQ==.jl:1
Arguments
  #self#::Core.Const(Main.foo)
  x::Int64
Body::Int64
1 ─ %1  = Main.:-::Core.Const(-)
│   %2  = Main.:+::Core.Const(+)
│   %3  = Main.:^::Core.Const(^)
│   %4  = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %5  = (%4)()::Core.Const(Val{2}())
│   %6  = Base.literal_pow(%3, x, %5)::Int64
│   %7  = Main.:*::Core.Const(*)
│   %8  = (%7)(3, x)::Int64
│   %9  = (%2)(%6, %8)::Int64
│   %10 = (%1)(%9, 1)::Int64
└──       return %10



In [6]:
@code_llvm foo(1)

; Function Signature: foo(Int64)
;  @ /home/csimal/Documents/Talks/CeCI/CECI-Julia-for-HPC/code/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W2sZmlsZQ==.jl:1 within `foo`
define i64 @julia_foo_11415(i64 signext %"x::Int64") #0 {
top:
;  @ /home/csimal/Documents/Talks/CeCI/CECI-Julia-for-HPC/code/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W2sZmlsZQ==.jl:2 within `foo`
; ┌ @ int.jl:87 within `+`
   %0 = add i64 %"x::Int64", 3
   %1 = mul i64 %0, %"x::Int64"
; └
; ┌ @ int.jl:86 within `-`
   %2 = add i64 %1, -1
   ret i64 %2
; └
}


## Array views
When modifying arrays slice by slice, it's often handy to use `views` which are essentially "sub-arrays" that are directly bound to the same underlying memory, while having adapted indices.

In [7]:
function inplace!(a)
    for i in eachindex(a)
        a[i] = a[i]^2
    end
end

inplace! (generic function with 1 method)

In [8]:
x = rand(5)

5-element Vector{Float64}:
 0.11144056285038484
 0.33243058187466434
 0.027166535095138777
 0.1918066135286458
 0.5078639846225823

In [9]:
inplace!(x)
x

5-element Vector{Float64}:
 0.012418999048410574
 0.11051009176552791
 0.0007380206290754068
 0.03678977699332729
 0.2579258268767265

In [10]:
A = rand(5,5)

5×5 Matrix{Float64}:
 0.142352  0.324387   0.206145   0.272745   0.971719
 0.816253  0.0771701  0.579599   0.562823   0.74918
 0.773032  0.962721   0.0857288  0.76549    0.0547727
 0.429408  0.4843     0.33207    0.0205874  0.237885
 0.195748  0.18239    0.459602   0.284693   0.041879

In [11]:
for i in axes(A,1)
    inplace!(A[i,:])
end
A

5×5 Matrix{Float64}:
 0.142352  0.324387   0.206145   0.272745   0.971719
 0.816253  0.0771701  0.579599   0.562823   0.74918
 0.773032  0.962721   0.0857288  0.76549    0.0547727
 0.429408  0.4843     0.33207    0.0205874  0.237885
 0.195748  0.18239    0.459602   0.284693   0.041879

`A` hasn't changed! That's because `A[i,:]` creates a copy.

In order to get the correct behavior, we need to use views.

In [12]:
for i in axes(A,1)
    inplace!(view(A, i, :))
end
A

5×5 Matrix{Float64}:
 0.020264   0.105227    0.0424959   0.0743896   0.944238
 0.666269   0.00595522  0.335934    0.316769    0.56127
 0.597578   0.926832    0.00734942  0.585975    0.00300005
 0.184392   0.234547    0.11027     0.00042384  0.0565893
 0.0383174  0.0332662   0.211234    0.0810501   0.00175385

Equivalently, we can use the `@views` macro, which will apply to every indexing operation within a block

In [13]:
 @views for i in axes(A,1)
    inplace!(A[i,:])
 end
 A

5×5 Matrix{Float64}:
 0.00041063  0.0110728   0.0018059  0.00553381  0.891586
 0.443915    3.54646e-5  0.112852   0.100343    0.315024
 0.3571      0.859017    5.4014e-5  0.343366    9.00028e-6
 0.0340003   0.0550123   0.0121596  1.79641e-7  0.00320235
 0.00146822  0.00110664  0.04462    0.00656912  3.07599e-6

## Type Stability

A function is said to be *type-stable* if its return type can be inferred from the types of its arguments. Type stability is pretty important, as its absence forces the Julia compiler to be more conservative. Any type unstable function will in general hurt performance, so they should be avoided at all costs.

The following function is type unstable, as the type of its output depends on the *value* of its input, not just its type.

In [14]:
function type_unstable(x)
    if x < 0
        return "Negative number"
    else
        return x
    end
end

type_unstable (generic function with 1 method)

A helpful tool to hunt for type instability is the `@code_warntype` macro.

In [15]:
@code_warntype type_unstable(1)

MethodInstance for type_unstable(::Int64)
  from type_unstable(x) @ Main ~/Documents/Talks/CeCI/CECI-Julia-for-HPC/code/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:1
Arguments
  #self#::Core.Const(Main.type_unstable)
  x::Int64
Body::Union{Int64, String}
1 ─ %1 = Main.:<::Core.Const(<)
│   %2 = (%1)(x, 0)::Bool
└──      goto #3 if not %2
2 ─      return "Negative number"
3 ─ %5 = x::Int64
└──      return %5



A more subtle example of type instability is when handling arithmetic expression. Julia tends to avoid implicit type conversions, which can sometimes lead to surprises. The following function is type unstable. Can you spot why?

In [16]:
relu(x) = x < 0 ? 0 : x

relu (generic function with 1 method)

In [17]:
relu(1.0), relu(-1.0)

(1.0, 0)

In [18]:
@code_warntype relu(-1.0)

MethodInstance for relu(::Float64)
  from relu(x) @ Main ~/Documents/Talks/CeCI/CECI-Julia-for-HPC/code/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X32sZmlsZQ==.jl:1
Arguments
  #self#::Core.Const(Main.relu)
  x::Float64
Body::Union{Float64, Int64}
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─ %4 = x::Float64
└──      return %4



The problem here is that the ternary operator returns 0 of type `Int` regardless of the type of `x`. In order to correct this, we can use the function `zero` which will return the zero value of the same type as `x`. Other functions like that include `one(x)` and `oftype(x,y)`

In [19]:
relu(x) = x < 0 ? zero(x) : x

relu (generic function with 1 method)

In [20]:
@code_warntype relu(-1.0)

MethodInstance for relu(::Float64)
  from relu(x) @ Main ~/Documents/Talks/CeCI/CECI-Julia-for-HPC/code/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X36sZmlsZQ==.jl:1
Arguments
  #self#::Core.Const(Main.relu)
  x::Float64
Body::Float64
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.zero::Core.Const(zero)
│   %4 = (%3)(x)::Core.Const(0.0)
└──      return %4
3 ─ %6 = x::Float64
└──      return %6



Another common issue with types is when incorrectly using abstract types. For example, the following type definition is particularly problematic

In [21]:
struct BadFoo
    x # this amounts to x::Any
    y::Real
    z::Vector{Integer}
end

Why is this bad? Well, because whenever a struct has abstract field types, the Julia compiler can't tell what it will hold, so it has to be ready for anything, which means using pointers to values, rather than the values themselves.

The correct way to do this is to use a parametric type, so that for any particular instance of our type, the compiler knows exactly the types of each field.

In [22]:
struct GoodFoo{T1,T2<:Real,T3<:Integer}
    x::T1
    y::T2
    z::Vector{T3}
end

As an example of how bad abstract field types (or element types can be) let's look at the generated code with `@code_llvm`.

In [23]:
mutable struct Bar{T<:AbstractFloat}
    a::T
end

In [24]:
foo(b::Bar) = m.a + 1

foo (generic function with 2 methods)

In [25]:
code_llvm(foo, Tuple{Float64})

; Function Signature: foo(Float64)
;  @ /home/csimal/Documents/Talks/CeCI/CECI-Julia-for-HPC/code/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W2sZmlsZQ==.jl:1 within `foo`
define double @julia_foo_13381(double %"x::Float64") #0 {
top:
;  @ /home/csimal/Documents/Talks/CeCI/CECI-Julia-for-HPC/code/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W2sZmlsZQ==.jl:2 within `foo`
; ┌ @ intfuncs.jl:370 within `literal_pow`
; │┌ @ float.jl:493 within `*`
    %0 = fmul double %"x::Float64", %"x::Float64"
; └└
; ┌ @ promotion.jl:430 within `*` @ float.jl:493
   %1 = fmul double %"x::Float64", 3.000000e+00
; └
; ┌ @ float.jl:491 within `+`
   %2 = fadd double %0, %1
; └
; ┌ @ promotion.jl:431 within `-` @ float.jl:492
   %3 = fadd double %2, -1.000000e+00
   ret double %3
; └
}


In [26]:
code_llvm(foo, Tuple{AbstractFloat})

; WARNING: This code may not match what actually runs.
; Function Signature: foo(AbstractFloat)
;  @ /home/csimal/Documents/Talks/CeCI/CECI-Julia-for-HPC/code/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W2sZmlsZQ==.jl:1 within `foo`
define nonnull ptr @julia_foo_13387(ptr noundef nonnull readonly %"x::AbstractFloat") #0 {
top:
  %jlcallframe1 = alloca [2 x ptr], align 8
  %gcframe2 = alloca [4 x ptr], align 16
  call void @llvm.memset.p0.i64(ptr align 16 %gcframe2, i8 0, i64 32, i1 true)
  %unionalloca.sroa.0 = alloca double, align 8
  %thread_ptr = call ptr asm "movq %fs:0, $0", "=r"() #12
  %tls_ppgcstack = getelementptr i8, ptr %thread_ptr, i64 -8
  %tls_pgcstack = load ptr, ptr %tls_ppgcstack, align 8
  store i64 8, ptr %gcframe2, align 16
  %frame.prev = getelementptr inbounds ptr, ptr %gcframe2, i64 1
  %task.gcstack = load ptr, ptr %tls_pgcstack, align 8
  store ptr %task.gcstack, ptr %frame.prev, align 8
  store ptr %gcframe2, ptr %tls_pgcstack, align 8
;  @ /home/csimal/

Notice how the second one is so much longer? That's because of all the checks the compiler has to do because it's unsure about the type. This, by the way is how Python operates by default.